In [18]:
import implicit_skill_finder
import utils
import pandas as pd
import numpy as np
from joblib import dump, load

In [19]:
def show_topics(vectorizer, lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [20]:
# Please change the input here
with open('usertest.txt', encoding='utf-8') as f:
    texts = [line for line in f]
for x in texts:
    print(type(x))

<class 'str'>
<class 'str'>
<class 'str'>


Find top 15 words of each topics. These popular words is selected as candidate for choosing skill list.

In [21]:
best_model = load(utils.MODEL_DIR)
tf_vectorizer = load(utils.VECTOR_DIR)
lda_output = load(utils.OUTPUT_DIR)
topic_keywords = show_topics(vectorizer=tf_vectorizer, lda_model=best_model, n_words=15)        
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

In [22]:
df_document_topic = pd.DataFrame(np.round(lda_output, 2))
topic_pos = (-df_document_topic.values).argsort()

In [23]:
for t in range(len(texts)):
    print(f'Processing text {t}')
    doc_ids, docs = implicit_skill_finder.find_similar_texts([texts[t]])
    implicit_skills = set()
    for i in doc_ids:
        for j in range(len(df_topic_keywords)):
            if topic_pos[i, j] > 0.3:
                implicit_skills.update(df_topic_keywords.iloc[j, :].apply(lambda x: x.strip()).to_list())
    print('Potential implicit skills are ' + ', '.join(implicit_skills))

Processing text 0
Potential implicit skills are support, product, digital, market, process, engineer, marketing, team, application, design, service, sale, partner, drive, pron, business, requirement, development, ensure, engineering, management, platform, technology, build, job, code, work, customer, project, software, campaign, experience, strategy, skill
Processing text 1
Potential implicit skills are support, product, digital, market, engineer, client, marketing, team, application, people, design, pron pron, service, sale, partner, provide, drive, pron, business, development, engineering, platform, opportunity, technology, build, code, work, customer, software, campaign, experience, strategy, skill
Processing text 2
Potential implicit skills are support, product, digital, market, engineer, client, marketing, team, application, people, design, pron pron, service, sale, partner, provide, drive, pron, business, development, engineering, platform, opportunity, technology, build, code, w